# Check intertrials for clean segments

This notebook checks to see whether any segments in the intertrial periods have been marked as clean and usable. This would be problematic because we need to only extract clean data to build our PSDs with from the trials.

In [1]:
import os
import sys
import glob

import xmltodict

def get_filelist(import_path, extension):
  """ 
  Returns list of file paths from import_path with specified extension.
  """
  filelist = []
  for root, dirs, files in os.walk(import_path):
      filelist += glob.glob(os.path.join(root, '*.' + extension))
      return filelist

def get_event_file(filepath, extract_all=False, extract_clean_segs=True):
    events = {}
    xmlevents = xmltodict.parse(open(filepath).read())
    xmlevents = xmlevents['EMSE_Event_List']['Event']
    n = 0
    for i in range(len(xmlevents)):
        if extract_all:
            events[i] = {'type': '', 'latency': 0, 'urevent': 0}
            events[i]['type'] = xmlevents[i]['Name']
            events[i]['latency'] = int(xmlevents[i]['Start'])
            events[i]['urevent'] = i
            # if (extract_clean_segs == True and xmlevents[i]['Name'] in ['C', 'O']) or\
        elif extract_all == False and\
             (len(xmlevents[i]['Name']) == 3 and xmlevents[i]['Name'] not in ['255', '222', '223', '252']):
            events[n] = {'type': '', 'latency': 0, 'urevent': 0}
            events[n]['type'] = xmlevents[i]['Name']
            events[n]['latency'] = int(xmlevents[i]['Start'])
            events[n]['urevent'] = n
            n += 1
    return events

def print_evt_information(filepath, print_in_secs=True):
    """
    Prints general information about a specified .evt file.
    Inputs:
        filepath: String, specifies full path to a .evt file.
        print_clean_segs: Boolean, specifies whether to print information regarding
                          the segments that have been marked as being clean.
        print_in_secs: Boolean, specifies whether to print .evt information in seconds,
                       as opposed to timepoints. Assumes 512 sampling rate.
    """
    events = get_event_file(filepath)
    eyesc_trials = 0
    eyeso_trials = 0
    eyesc_trial_length = 0
    eyeso_trial_length = 0
    for i in range(len(events)-1):
        # Eyes-closed trial
        if events[i]['type'][0:2] == '10' and events[i+1]['type'][0:2] == '20':
            eyesc_trials += 1
            eyesc_trial_length += events[i+1]['latency'] - events[i]['latency']
        # Eyes-open trial
        elif events[i]['type'][0:2] == '11' and events[i+1]['type'][0:2] == '21':
            eyeso_trials += 1
            eyeso_trial_length += events[i+1]['latency'] - events[i]['latency']
    eyesc_trial_length /= eyesc_trials
    eyeso_trial_length /= eyeso_trials
    if print_in_secs:
        eyesc_trial_length /= 512;
        eyeso_trial_length /= 512;

    print('{0} | eyesc_trials: {1} | eyeso_trials: {2} | eyesc_trial_length: {3} | eyeso_trial_length: {4}'.format(\
        filepath.split('/')[-1], eyesc_trials, eyeso_trials, eyesc_trial_length, eyeso_trial_length))

import_path = '/Users/jorge/Drive/research/_psd-slope/data/rs/full/evt/'
extension = 'evt'
# files = get_filelist(import_path, extension)
# for file in files:
#     print_evt_information(file)

files = get_filelist(import_path, extension)
for file in files:
    print_evt_information(file)

1121181181.evt | eyesc_trials: 4 | eyeso_trials: 4 | eyesc_trial_length: 59.99853515625 | eyeso_trial_length: 59.99853515625
1121181183.evt | eyesc_trials: 4 | eyeso_trials: 4 | eyesc_trial_length: 59.99853515625 | eyeso_trial_length: 59.998046875
1121181218.evt | eyesc_trials: 4 | eyeso_trials: 4 | eyesc_trial_length: 59.998046875 | eyeso_trial_length: 59.99755859375
1121181262.evt | eyesc_trials: 4 | eyeso_trials: 4 | eyesc_trial_length: 60.0068359375 | eyeso_trial_length: 59.97119140625
1121181286.evt | eyesc_trials: 4 | eyeso_trials: 4 | eyesc_trial_length: 59.998046875 | eyeso_trial_length: 59.78466796875
112118131.evt | eyesc_trials: 4 | eyeso_trials: 4 | eyesc_trial_length: 59.998046875 | eyeso_trial_length: 59.998046875
1121181334.evt | eyesc_trials: 4 | eyeso_trials: 4 | eyesc_trial_length: 60.015625 | eyeso_trial_length: 60.0068359375
112118135.evt | eyesc_trials: 4 | eyeso_trials: 4 | eyesc_trial_length: 59.998046875 | eyeso_trial_length: 59.998046875
1121181393.evt | eyesc_